In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.data import Dataset
import pandas as pd
from os.path import join

2023-12-03 19:22:11.030347: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-03 19:22:11.030374: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-03 19:22:11.031986: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-03 19:22:11.141431: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [3]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [4]:
tf.keras.backend.set_image_data_format('channels_last')

In [5]:
tf.test.is_gpu_available(
    cuda_only=False, min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-12-03 19:22:17.875483: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


True

2023-12-03 19:22:17.902554: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:22:17.902699: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:22:18.830323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:
model = sm.Unet('resnet34', classes=1, input_shape=(1024, 1024, 3), activation='sigmoid')

2023-12-03 19:22:21.216576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:22:21.216885: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-03 19:22:21.217062: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [7]:
df_file_name_split = pd.read_csv('../data/train_unique_cropped/file_name_split.csv')
df_file_name_split.head()

,file_path,dataset
0,positive/4cbc6a0a0_256_512_384_640.jpg,train
1,positive/4cbc6a0a0_384_128_512_256.jpg,train
2,positive/4cbc6a0a0_640_0_768_128.jpg,train
3,positive/4cbc6a0a0_128_512_256_640.jpg,train
4,positive/4cbc6a0a0_0_512_128_640.jpg,train


In [8]:
image_path_list_train = df_file_name_split[df_file_name_split['dataset'] == 'train']['file_path']
image_path_list_val = df_file_name_split[df_file_name_split['dataset'] == 'val']['file_path']
len(image_path_list_train), len(image_path_list_val)

(8671, 289)

In [9]:
image_full_path_list_train = [(join('../data/train_unique_cropped/', p), join('../data/train_unique_cropped/', p.replace('.jpg', '.png'))) 
                              for p in image_path_list_train]
image_full_path_list_val = [(join('../data/train_unique_cropped/', p), join('../data/train_unique_cropped/', p.replace('.jpg', '.png'))) 
                            for p in image_path_list_val]

In [10]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

def parse_image(filename):
    image = tf.io.read_file(filename[0])
    image = tf.io.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [1024, 1024])
    image = preprocess_input(image)

    mask = tf.io.read_file(filename[1])
    mask = tf.io.decode_png(mask)
    mask = tf.image.convert_image_dtype(mask, tf.float32)
    mask = tf.image.resize(mask, [1024, 1024])
    
    return image, mask

dataset_train = (Dataset.from_tensor_slices(image_full_path_list_train)
                 .repeat()
                 .shuffle(buffer_size=len(image_full_path_list_train), seed=42, reshuffle_each_iteration=True)
                 .map(parse_image)
                 .batch(4, drop_remainder=True))
dataset_val = (Dataset.from_tensor_slices(image_full_path_list_val)
               .repeat()
               .shuffle(buffer_size=len(image_full_path_list_val), seed=42, reshuffle_each_iteration=True)
               .map(parse_image)
               .batch(4,  drop_remainder=True))

2023-12-03 19:22:25.603322: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [ ]:
# define model
model = sm.Unet(BACKBONE, classes=1, encoder_weights='imagenet')
model.compile(
    'Adam',
    loss=sm.losses.binary_crossentropy,
    metrics=[sm.metrics.iou_score],
)

model.fit(
    x=dataset_train,
    steps_per_epoch=3,
    validation_data=dataset_val,
    validation_steps=2,
    epochs=1,
    verbose=2,
    shuffle=False,
)

2023-12-03 19:37:43.423176: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2023-12-03 19:37:43.500405: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2023-12-03 19:37:43.500448: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2023-12-03 19:37:44.078735: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2023-12-03 19:37:44.087175: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 16777216 exceeds 10% of free system memory.
2023-12-03 19:37:48.856208: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
